In [1]:
%pip install -q pandas numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import os
import functions.preprocessing as prep
import gc

Load the original dataset

In [3]:
dataset_path = os.path.join(os.getcwd(),'data/training_set_VU_DM.csv')

#change this to increase size of training data
no_of_rows_to_get = 10000000

train_df_og = pd.read_csv(dataset_path,chunksize=no_of_rows_to_get)

train_df_og = train_df_og.get_chunk(no_of_rows_to_get)

train_df_og.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,893,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,10404,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,21315,3,4.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,27348,2,4.0,...,NaN,NaN,NaN,NaN,-1.0,0.0,5.0,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,29604,4,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


New Way of Balancing

In [4]:
book_and_clicked_grp = train_df_og[(train_df_og['booking_bool']==1) & (train_df_og['click_bool']==1)]
clicked_grp = train_df_og[(train_df_og['booking_bool']==0) & (train_df_og['click_bool']==1)]
nothing_grp = train_df_og[(train_df_og['booking_bool']==0) & (train_df_og['click_bool']==0)]

size_most_rel = len(book_and_clicked_grp)
size_rel = len(clicked_grp)
size_least_rel = len(nothing_grp)

print(size_most_rel, size_rel, size_least_rel)

138390 83489 4736468


In [5]:
sample_size_mr = size_most_rel
sample_size_r = size_rel
sample_size_lr = int(2*(sample_size_mr+sample_size_r))

book_and_clicked_grp = book_and_clicked_grp.sample(sample_size_mr,random_state=7).reset_index(drop=True)
clicked_grp = clicked_grp.sample(sample_size_r,random_state=7).reset_index(drop=True)
nothing_grp = nothing_grp.sample(sample_size_lr,random_state=7).reset_index(drop=True)

In [6]:
train_data = pd.concat([book_and_clicked_grp,clicked_grp,nothing_grp]).reset_index(drop=True)
train_data.sort_values(by='srch_id',ascending=True,inplace=True,ignore_index=True)

In [7]:
print(len(train_data))
train_data.head()


665637


,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,68914,2,3.0,...,NaN,NaN,NaN,NaN,0.0,0.0,11.0,1,114.29,1
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,56880,4,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,81437,3,4.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,88127,3,3.0,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,44147,3,3.5,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,0,NaN,0


In [8]:
del [[train_df_og,book_and_clicked_grp,clicked_grp,nothing_grp]]
gc.collect()
train_df_og = pd.DataFrame()
book_and_clicked_grp = pd.DataFrame()
clicked_grp = pd.DataFrame()
nothing_grp = pd.DataFrame()

Balancing the Dataset (Ignore these markdowns)

df1 = train_df_og.groupby('srch_id')["booking_bool"].apply(lambda x: (x==1).sum()).reset_index(name='booked')
df2 = df1.groupby('booked').count()
print("Number of Searches (Booked vs Non-Booked) : ")
df2

all_srch_df = df1.groupby('booked')
df_booked = all_srch_df.get_group(1).drop("booked",axis=1)
df_not_booked = all_srch_df.get_group(0).drop("booked",axis=1)

len_df = [len(df_not_booked),len(df_booked)]
lower_number = len_df[np.argmin(len_df)]
print(np.argmin(len_df))
print(lower_number)

no_of_non_booked = np.min([lower_number,20000])
no_of_booked = int(1*no_of_non_booked)
no_of_booked = np.min([no_of_booked,len(df_booked)])
print(no_of_non_booked)
print(no_of_booked)

df_booked = df_booked.sample(n=no_of_booked,random_state=7).reset_index(drop=True)
df_not_booked = df_not_booked.sample(n=no_of_non_booked,random_state=10).reset_index(drop=True)

train_data_booked = train_df_og.merge(df_booked,on='srch_id')
train_data_not_booked = train_df_og.merge(df_not_booked,on='srch_id')

train_data = pd.concat([train_data_not_booked,train_data_booked]).reset_index(drop=True)

train_data.head()

len(train_data)

del[[train_df_og,df_booked,df_not_booked,train_data_booked,train_data_not_booked]]
gc.collect()
train_df_og = pd.DataFrame()
df_booked = pd.DataFrame()
df_not_booked = pd.DataFrame()
train_data_booked = pd.DataFrame()
train_data_not_booked = pd.DataFrame()

Adding New Features (Imp Part starts from here)

Merging Comp Stats across all rows

In [9]:
train_data = prep.mergeCompsAll(train_data)
train_data = prep.fill_comp_rate(train_data)
train_data = prep.fill_comp_inv(train_data)
train_data = prep.fill_rate_diff(train_data)
train_data.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,comp_rate,comp_inv,comp_rate_percent_diff
0,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,68914,2,3.0,...,NaN,0.0,0.0,11.0,1,114.29,1,0.0,0.0,11.0
1,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,56880,4,4.0,...,NaN,0.0,0.0,NaN,0,NaN,0,0.0,0.0,0.0
2,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,81437,3,4.0,...,NaN,0.0,0.0,NaN,0,NaN,0,0.0,0.0,0.0
3,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,88127,3,3.0,...,NaN,0.0,0.0,NaN,0,NaN,0,0.0,0.0,0.0
4,1,2013-04-04 08:32:15,12,187,NaN,NaN,219,44147,3,3.5,...,NaN,0.0,0.0,NaN,0,NaN,0,0.0,0.0,0.0


Adding Relevancy Column

In [10]:
conditions = [
    (train_data['booking_bool'] == 1),
    (train_data['click_bool'] == 1 ) & (train_data['booking_bool'] == 0),
    (train_data['click_bool'] == 0),
    ]
values = ['5', '1', '0']
train_data['relevancy'] = np.select(conditions, values)
train_data['relevancy'] = train_data['relevancy'].astype('category')

train_data['relevancy'].value_counts()

0    443758
5    138390
1     83489
Name: relevancy, dtype: int64

Adding Visitor+Prop same country Column

In [11]:
conditions = [
    (train_data['visitor_location_country_id'] == train_data['prop_country_id']),
    (train_data['visitor_location_country_id'] != train_data['prop_country_id'])
]

values = ['1','0']
train_data['same_country'] = np.select(conditions,values)

train_data['same_country'].value_counts()

1    422499
0    243138
Name: same_country, dtype: int64

Selecting Features required for training in output csv

In [12]:
common_columns_train_test = ['site_id','visitor_location_country_id','prop_country_id','prop_starrating','prop_review_score','prop_brand_bool','prop_location_score1','prop_location_score2','prop_log_historical_price','promotion_flag','srch_destination_id','srch_length_of_stay','srch_booking_window','srch_adults_count','srch_children_count','srch_room_count','srch_saturday_night_bool','random_bool','comp_rate','comp_inv','comp_rate_percent_diff','same_country']

In [13]:
columns_train = common_columns_train_test+['relevancy']

train_data = train_data[columns_train]

train_data.fillna({'prop_review_score':train_data['prop_review_score'].mean()},inplace=True)
train_data.fillna({'prop_location_score2':train_data['prop_location_score2'].mean()},inplace=True)


train_data.head()

,site_id,visitor_location_country_id,prop_country_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,promotion_flag,...,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,comp_rate,comp_inv,comp_rate_percent_diff,same_country,relevancy
0,12,187,219,2,3.0,1,2.20,0.0206,4.44,0,...,4,0,1,1,1,0.0,0.0,11.0,0,5
1,12,187,219,4,4.0,1,2.83,0.1028,5.15,0,...,4,0,1,1,1,0.0,0.0,0.0,0,0
2,12,187,219,3,4.0,1,2.30,0.0155,5.03,0,...,4,0,1,1,1,0.0,0.0,0.0,0,0
3,12,187,219,3,3.0,1,1.39,0.0038,4.80,0,...,4,0,1,1,1,0.0,0.0,0.0,0,0
4,12,187,219,3,3.5,1,2.20,0.0356,4.81,0,...,4,0,1,1,1,0.0,0.0,0.0,0,0


Outputting to disk

In [14]:
out_path = os.path.join(os.getcwd(),'data/train_data.csv')
train_data.to_csv(path_or_buf=out_path)

In [15]:
del[[train_data]]
gc.collect()
train_data = pd.DataFrame()

Preprocessing Test Data Set

In [16]:
test_set_path = os.path.join(os.getcwd(),'data/test_set_VU_DM.csv')

no_of_test_rows = 10000000

test_df_og = pd.read_csv(test_set_path,chunksize=no_of_test_rows)

test_df_og = test_df_og.get_chunk(no_of_test_rows)

test_df_og.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff
0,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,3180,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,5543,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,14142,2,3.5,...,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,22393,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,24194,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Adding New Features (same as training set)

In [17]:
test_data = prep.mergeCompsAll(test_df_og)
test_data = prep.fill_comp_rate(test_data)
test_data = prep.fill_comp_inv(test_data)
test_data = prep.fill_rate_diff(test_data)
test_data.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,comp_rate,comp_inv,comp_rate_percent_diff
0,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,3180,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
1,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,5543,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
2,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,14142,2,3.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,10.0
3,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,22393,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
4,1,2013-02-02 15:27:40,24,216,NaN,NaN,219,24194,3,4.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [18]:
del[[test_df_og]]
gc.collect()
test_df_og = pd.DataFrame()

In [19]:
conditions = [
    (test_data['visitor_location_country_id'] == test_data['prop_country_id']),
    (test_data['visitor_location_country_id'] != test_data['prop_country_id'])
]

values = ['1','0']
test_data['same_country'] = np.select(conditions,values)

test_data['same_country'].value_counts()

1    62444
0    37556
Name: same_country, dtype: int64

Selecting Features for final test set

In [20]:
test_columns = common_columns_train_test+['srch_id','prop_id']

test_data = test_data[test_columns]

test_data.fillna({'prop_review_score':test_data['prop_review_score'].mean()},inplace=True)
test_data.fillna({'prop_location_score2':test_data['prop_location_score2'].mean()},inplace=True)


test_data.head()

,site_id,visitor_location_country_id,prop_country_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,promotion_flag,...,srch_children_count,srch_room_count,srch_saturday_night_bool,random_bool,comp_rate,comp_inv,comp_rate_percent_diff,same_country,srch_id,prop_id
0,24,216,219,3,4.5,1,2.94,0.0691,5.03,0,...,0,1,0,0,0.0,0.0,0.0,0,1,3180
1,24,216,219,3,4.5,1,2.64,0.0843,4.93,0,...,0,1,0,0,0.0,0.0,0.0,0,1,5543
2,24,216,219,2,3.5,1,2.71,0.0556,4.16,0,...,0,1,0,0,1.0,0.0,10.0,0,1,14142
3,24,216,219,3,4.5,1,2.40,0.0561,5.03,0,...,0,1,0,0,0.0,0.0,0.0,0,1,22393
4,24,216,219,3,4.5,1,2.94,0.2090,4.72,0,...,0,1,0,0,0.0,0.0,0.0,0,1,24194


Outputting to disk

In [21]:
out_path = os.path.join(os.getcwd(),'data/test_data.csv')
test_data.to_csv(path_or_buf=out_path)